In [322]:
import time
import os
import json
import datetime

def get_timestamp():
    return float("{0:.2f}".format(time.time()))
    # return strftime("%Y-%m-%d %H:%M:%S", gmtime())

def timestamp_ealier_than(begin, days):
    return int((begin - datetime.timedelta(days=days)).strftime("%s"))

class sql_conn:
    def __init__(self, conn):
        self.conn = conn
        self.cursor = conn.cursor()

    def __search_user_by_name(self, username):
        self.cursor.execute("select * from users where username='{}';".format(username))
        return self.cursor.fetchone()  # None if it doesn't exist

    def __search_user_by_id(self, userid):
        self.cursor.execute("select * from users where userid={};".format(userid))
        return self.cursor.fetchone()  # None if it doesn't exist

    def __search_admin_by_name(self, adminname):
        result = self.cursor.execute("select * from admin where adminname='{}';".format(adminname))
        return self.cursor.fetchone()  # None if it doesn't exist

    def __search_source_by_name(self, sourcename):
        result = self.cursor.execute("select * from source where sourcename='{}';".format(sourcename))
        return self.cursor.fetchone()  # None if it doesn't exist

    def __search_source_by_id(self, sourceid):
        result = self.cursor.execute("select * from source where sourceid='{}';".format(sourceid))
        return self.cursor.fetchone()  # None if it doesn't exist

    def __insertion(self, sql):
        try:
            self.cursor.execute(sql)
            self.conn.commit()
            return 1
        except:
            return -1

    def __exe_sql(self, sql):
        try:
            self.cursor.execute(sql)
            return self.cursor.fetchall()
        except:
            return None

    def __get_by_option(self, tablename, target_col, col_val_dict, head=True):
        # search by one of the condition
        try:
            for col in col_val_dict.keys():
                if col_val_dict[col] != None:
                    pkcol = col
            self.cursor.execute(
                "select {} from {} where {}='{}';".format(target_col, tablename, pkcol, col_val_dict[pkcol]))
            if head:
                return self.cursor.fetchone()[0]
            else:
                return self.cursor.fetchone()
        except:
            return None

    def __get_by_mul_cond(self, tablename, target_col, col_val_dict):
        # search by multiple conditions
        try:
            sql = "select {} from {} where ".format(target_col, tablename)
            for i, col in enumerate(col_val_dict.keys()):
                if i != 0:
                    sql += " and "
                sql += "{} = '{}'".format(col, col_val_dict[col])

            sql += ";"
            # print(sql)
            self.cursor.execute(sql)
            return self.cursor.fetchone()[0]
        except:
            return None

    # user*******************************************************************************
    def get_all_user(self):
        return self.__exe_sql("select * from users;")

    def get_user(self, userid=None, username=None, user_email=None):
        return self.__get_by_option('users', '*', {'userid': userid, 'username': username, 'email_address': user_email},
                                    head=False)

    def get_user_number(self):
        return self.__exe_sql("select count(*) from users;")[0][0]

    def get_user_id(self, username=None, user_email=None):
        return self.__get_by_option('users', 'userid', {'username': username, 'email_address': user_email})

    def get_user_passwd(self, userid=None, username=None, user_email=None):
        return self.__get_by_option('users', 'password',
                                    {'userid': userid, 'username': username, 'email_address': user_email})

    def get_user_email(self, userid=None, username=None):
        return self.__get_by_option('users', 'email_address', {'userid': userid, 'username': username})

    def get_user_name(self, userid=None, user_email=None):
        return self.__get_by_option('users', 'username', {'userid': userid, 'email_address': user_email})

    def get_user_credit(self, userid=None, username=None, user_email=None):
        return self.__get_by_option('users', 'credits',
                                    {'userid': userid, 'username': username, 'email_address': user_email})

    def get_user_nb_accept(self, userid=None, username=None, user_email=None):
        return self.__get_by_option('users', 'nb_accept',
                                    {'userid': userid, 'username': username, 'email_address': user_email})

    def get_user_nb_answer(self, userid=None, username=None, user_email=None):
        return self.__get_by_option('users', 'nb_answer',
                                    {'userid': userid, 'username': username, 'email_address': user_email})

    def get_user_nb_val(self, userid=None, username=None, user_email=None):
        return self.__get_by_option('users', 'nb_val',
                                    {'userid': userid, 'username': username, 'email_address': user_email})

    def get_user_nb_val_tp(self, userid=None, username=None, user_email=None):
        return self.__get_by_option('users', 'nb_val_tp',
                                    {'userid': userid, 'username': username, 'email_address': user_email})

    def get_user_val_acc(self, userid=None, username=None, user_email=None):
        total = self.get_user_nb_val(userid, username, user_email)
        if total != 0:
            return self.get_user_nb_val_tp(userid, username, user_email) / total
        else:
            return 0

    def get_user_accept_rate(self, userid=None, username=None, user_email=None):
        total = self.get_user_nb_answer(userid, username, user_email)
        if total != 0:
            return self.get_user_nb_accept(userid, username, user_email) / self.get_user_nb_answer(userid, username,
                                                                                                   user_email)
        else:
            return 0


    def insert_user(self, username, user_email, passwd, signin_time=get_timestamp(), credits=0, nb_accept=0,
                    nb_answer=0, nb_examined=0):
        # insertion: 1 success, 0: already exist, -1: fail
        if self.__search_user_by_name(username) == None:
            sql = "INSERT INTO `se_proj`.`users` (`username`,`email_address`,`password`,`signin_date`,`credits`,`nb_accept`,`nb_answer`,`nb_examined`) VALUES ('{}','{}','{}','{}','{}',{},{},{});" \
                .format(username, user_email, passwd, signin_time, credits, nb_accept, nb_answer, nb_examined)
            return self.__insertion(sql)
        else:
            return 0

    def user_exist(self, userid=None, username=None, user_email=None):
        result = self.__get_by_option('users', '*',
                                      {'userid': userid, 'username': username, 'email_address': user_email})
        return True if result != None else False
    
    def get_user_credits_rank(self, user_email):
        userid = self.get_user_id(user_email=user_email)
        sql = "select i.rank from \
        (SELECT u.userid,\
        @curRank := @curRank + 1 AS rank  FROM  se_proj.users u, (SELECT @curRank := 0) r \
        ORDER BY  u.credits desc ) i where i.userid={};".format(userid)
        return int(self.__exe_sql(sql)[0][0])

    
    def get_user_source(self, user_email):
        userid = self.get_user_id(user_email=user_email)
        return self.__exe_sql("select distinct(td.datasource) from text_label tl \
        join text_data td on td.dataid=tl.dataid join users u  on tl.userid= u.userid where u.userid={};".format(userid))
    
    def get_user_mainpage_pan(self, user_email):
        try:
            # return [(priority, count), (priority, count), ...]
            userid = self.get_user_id(user_email=user_email)
            sql = "select x.priority, count(*) from \
            (select  distinct(td.datasource), s.priority from text_label tl \
            join text_data td on tl.dataid=td.dataid join source s on s.sourceid = td.datasource \
            where userid={}) x group by x.priority;".format(userid)
            return self.__exe_sql(sql)
        except:
            return [(1,0),(2,0),(3,0)]
    
    def __user_label_later_number(self, ts_list, tr):
        cnt = 0
        for ts in ts_list:
            if ts[0]>tr:
                cnt += 1
        return cnt
    def get_user_mainpage_pan_history(self, user_email):
        userid = self.get_user_id(user_email=user_email)
        sql = "select labeldate from text_label where userid={} order by labeldate desc;".format(userid) 
        ts_list = self.__exe_sql(sql)
        days_list = [1,3,7,30]
        result = []
        now = datetime.datetime.now()
        for d in days_list:
            ts_tr = timestamp_ealier_than(now, d) #threshold
            result.append(self.__user_label_later_number(ts_list, ts_tr))
            
        result.append(len(ts_list))
        return result
        
    # admin******************************************************************************
    def get_admin(self, adminid=None, adminname=None, admin_email=None):
        return self.__get_by_option('admin', '*',
                                    {'adminid': adminid, 'adminname': adminname, 'email_address': admin_email},
                                    head=False)

    def insert_admin(self, email_addr, adminname, passwd, access_level=1):
        # insertion: 1 success, 0: already exist, -1: fail
        if self.__search_admin_by_name(adminname) == None:
            sql = "INSERT INTO `se_proj`.`admin` (`email_address`,`adminname`,`password`,`access_level`) VALUES ('{}','{}','{}','{}');" \
                .format(email_addr, adminname, passwd, access_level)
            return self.__insertion(sql)
        else:
            return 0
        
    def get_admin_source(self, adminid=None, adminname=None, admin_email=None):
        if adminid==None:
            adminid = self.__get_by_option('admin', 'adminid', {'email_address': admin_email, 'adminname': adminname})
        return self.__exe_sql("select * from source where publisher={}".format(adminid))

    # source*****************************************************************************

    def get_source_number(self):
        return self.__exe_sql("select count(*) from source;")[0][0]

    def get_source(self, sourcename=None, sourceid=None):
        return self.__get_by_option('source', '*', {'sourceid': sourceid, 'sourcename': sourcename}, head=False)

    def get_all_source(self):
        return self.__exe_sql("select * from source;")

    def get_source_id(self, sourcename):
        return self.__get_by_option('source', 'sourceid', {'sourcename': sourcename})

    def insert_source(self, sourcename, finished=0, publisher='NULL', description='', publish_time=get_timestamp(),
                      priority=1):
        # insertion: 1 success, 0: already exist, -1: fail
        if self.__search_source_by_name(sourcename) == None:

            sql = "INSERT INTO `se_proj`.`source` (`sourcename`,`finished`,`publisher`,`description`,`publish_date`, `priority`)\
            VALUES ('{}',{}, {},'{}',{},{});".format(sourcename, finished, publisher, description, publish_time,
                                                     priority)
            # print(sql)
            return self.__insertion(sql)
        else:
            return 0
        
    def get_recent_source(self, limit=5):
        return self.__exe_sql("select * from source order by publish_date desc limit {};".format(limit))
    
    def get_source_by_priority(self, priority):
        # priority should be 1 2 or 3
        return self.__exe_sql("select * from se_proj.source where priority ={};".format(priority))

    # data *****************************************************************************
    def __insert_textdata(self, sourceid, data_index, data_path, final_labelid='NULL'):
        if self.__search_source_by_id(sourceid) != None:
            sql = "INSERT INTO `se_proj`.`text_data` (`datasource`,`data_index`,`data_path`,`final_labelid`) VALUES ({},{},'{}',{});" \
                .format(sourceid, data_index, data_path, final_labelid)
            return self.__insertion(sql)
        else:
            return 0

    def load_data(self, root_path, sourceid=None, sourcename=None):
        # load data(json file) from root folder into database
        sourceid = self.__get_by_option('source', 'sourceid', {'sourceid': sourceid, 'sourcename': sourcename})
        try:
            _, _, files = next(os.walk(root_path))
            self.__exe_sql("UPDATE `se_proj`.`source` SET `nb_json`= {}  WHERE `sourceid`={};".format(len(files), sourceid))
            for f in files:
                with open(os.path.join(root_path, f)) as js:
                    data_index = json.load(js)['index']
                if None != self.__get_by_mul_cond('text_data', 'dataid',
                                                  {'datasouce': sourceid, 'data_index': data_index}):
                    continue
                self.__insert_textdata(sourceid, data_index, os.path.join(root_path, f))
            return 1
        except:
            return 0

    def __get_textdata_sth(self, target_col, data_index, sourceid=None, sourcename=None):
        sourceid = self.__get_by_option('source', 'sourceid', {'sourceid': sourceid, 'sourcename': sourcename})
        return self.__get_by_mul_cond('text_data', target_col, {'datasource': sourceid, 'data_index': data_index})

    def get_textdataid(self, data_index, sourceid=None, sourcename=None):
        return self.__get_textdata_sth('dataid', data_index, sourceid, sourcename)


    def update_final_labelid(self, data_index, labelid, sourceid=None, sourcename=None):
        # 1:sucess -1:fail  0:souce or data not exist
        sourceid = self.__get_by_option('source', 'sourceid', {'sourceid': sourceid, 'sourcename': sourcename})
        dataid = self.get_textdataid(data_index, sourceid=sourceid)
        if sourceid != None and dataid != None:
            sql = "UPDATE `se_proj`.`text_data` SET `final_labelid`={} WHERE `dataid`={};".format(labelid, dataid)
            return self.__insertion(sql)
        else:
            return 0
    
    def fetch_data(self, sourcename, user_email, nb=5):
        userid=self.get_user_id(user_email=user_email)
        sourceid = self.get_source_id(sourcename=sourcename)
        
        l = self.__exe_sql("select td.dataid, td.datasource, td.data_index, td.data_path from text_data td \
        left join text_label tl on tl.dataid = td.dataid \
        where td.datasource ={} and td.final_labelid is NULL and (tl.userid!={} or tl.userid is NULL)\
        limit {};".format(sourceid, userid, nb))
        
        result = []
        for i in l:
            with open(i[-1]) as f:
                data = json.load(f)
            data['dataid'] = int(i[0])
            result.append(data)
        return result

    
    # label *****************************************************************************
    
    # index: dataid, userid
    def __get_label_sth(self, target_col, dataid, userid=None, username=None, user_email=None):
        if userid == None:
            userid = self.__get_by_option('users', 'userid', {'username': username, 'email_address': user_email})
        return self.__get_by_mul_cond('text_label', target_col, {'dataid': dataid, 'userid': userid})

    def get_label_correct(self, dataid, userid=None, username=None, user_email=None):
        # return 0 not determined, 1 correct, -1 not correct
        return self.__get_label_sth('correct', userid, username, user_email)

    def insert_label(self, user_email, json_list, save_dir='/home/se2018/label/', label_date=get_timestamp(), correct=0):
        # insert label , save label json file from the same user of the same project
        try:
            userid=self.get_user_id(user_email=user_email)
            proj_name = json_list[0]['projectName']
            save_dir = save_dir+'{}/'.format(proj_name)
            
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
                
            for j in json_list:
                #save the file
                save_path = save_dir+str(j['index'])+'_'+str(userid)+'_label.json'
                with open(save_path, 'w') as outfile:
                    json.dump(j, outfile)

                sourceid = self.get_source_id(sourcename=proj_name)
                label_content = []
                for subtask in j['task']:
                    label_content.append(subtask['label'])


                sql = "INSERT INTO text_label(`dataid`,`userid`,`labeldate`,`label_path`,`label_content`,`correct`) VALUES \
                ({},{},{},'{}','{}',{});".format(j['dataid'], userid, label_date, save_path, label_content, correct)
                print(sql)
                if(self.__insertion(sql)== -1):
                    return 0
            return 1
        except:
            return -1
    
    

    def close(self):
        self.cursor.close()
        self.conn.close()

In [328]:
import mysql.connector
from mysql.connector import connection
from IPython.core.debugger import set_trace
import os
import json

In [332]:
cnx = connection.MySQLConnection(user = 'root', password = 'se2018', host = '127.0.0.1', database = 'se_proj')

In [333]:
c = sql_conn(cnx)

In [334]:
c.get_user_mainpage_pan_history(user_email="xuch@sb.com")

[2, 2, 2, 3, 4]

In [144]:
c.get_user_mainpage_pan(user_email="jj@jj.com")

[]

In [120]:
c.get_user_source(user_email="huyx@sb.com")

[(13,)]

In [121]:
a = c.fetch_data(sourcename='nb_test', user_email='jiangtk@sb.com', nb=5)

In [122]:
a

[{'data': '谢丹宁真漂亮',
  'dataid': 45,
  'index': 1,
  'projectName': 'nb_test',
  'task': [{'aim': '请标出下面这句话的情感态度',
    'choices': ['Positive', 'Neutral', 'Negative'],
    'front': 'option',
    'label': '',
    'mode': 'single'},
   {'aim': '请选择这句话的情感：',
    'choices': ['开心', '满意', '失望', '生气'],
    'front': 'box',
    'label': ['开心', '满意'],
    'mode': 'multiple'},
   {'aim': '请选出句子中的形容词：', 'front': 'blank', 'label': [], 'mode': 'open'}]},
 {'data': '今天天气真好啊',
  'dataid': 38,
  'index': 2,
  'projectName': 'nb_test',
  'task': [{'aim': '请标出下面这句话的情感态度',
    'choices': ['Positive', 'Neutral', 'Negative'],
    'front': 'option',
    'label': '',
    'mode': 'single'},
   {'aim': '请选择这句话的情感：',
    'choices': ['开心', '满意', '失望', '生气'],
    'front': 'box',
    'label': [],
    'mode': 'multiple'},
   {'aim': '请选出句子中的形容词：', 'front': 'blank', 'label': [], 'mode': 'open'}]},
 {'data': '请问教学楼在哪',
  'dataid': 42,
  'index': 3,
  'projectName': 'nb_test',
  'task': [{'aim': '请标出下面这句话的情感态度',
    'cho

In [132]:
c.insert_label(user_email='xuch@sb.com', json_list=a)

INSERT INTO text_label(`dataid`,`userid`,`labeldate`,`label_path`,`label_content`,`correct`) VALUES                 (45,3,1528612323.28,'/home/se2018/label/nb_test/1_3_label.json','['', ['开心', '满意'], []]',0);


0

In [66]:
c.get_user_credits_rank(user_email="huyx@sb.com")

2

In [67]:
c.load_data(root_path='/home/se2018/data/xiednproj/', sourcename='xiednproj')

0

In [68]:
#test user
print(c.insert_user('test', 'test@sb.com', 'test', credits = 0))
print(c.get_user_id('jiangtk'))
print(c.get_user_passwd(username='test'))
print(c.get_user_passwd(userid=1))
print(c.get_user_credit(username='jiangtk'))
print(c.user_exist(username='huyx'))
print(c.user_exist(userid=1))
print(c.user_exist(userid=0))
print(c.get_user(username='huyx'))

0
1
test
jiangtk
19
True
True
False
(2, 'huyx@sb.com', 'huyx', 'huyx', None, 10, 0, 0, 0, 0)


In [74]:
# test admin
print(c.insert_admin('test@sb.com', 'test', 'test', access_level=1))
print(c.get_admin(adminname='xiedn'))
print(c.get_all_user())
print(c.get_user_number())

0
(1, 'xiedn@666.com', 'xiedn', 'xiedn', 2)
[(1, 'jiangtk@sb.com', 'jiangtk', 'jiangtk', None, 19, 15, 15, 5, 4), (2, 'huyx@sb.com', 'huyx', 'huyx', None, 10, 0, 0, 0, 0), (3, 'xuch@sb.com', 'xuch', 'xuch', None, 0, 0, 0, 0, 0), (12, 'test@sb.com', 'test', 'test', 1527145856.0, 0, 0, 0, 0, 0), (13, 'yxy@qq.com', 'yandadiao', 'yandadiao', 1527390848.0, 0, 0, 0, 0, 0), (14, 'huhu@qq.com', 'huhu', 'huhuhu', 1527397120.0, 0, 0, 0, 0, 0), (15, 'uu@uu.com', 'uu', 'uuu', 1527397632.0, 0, 0, 0, 0, 0), (16, 'uu@uuuu.com', 'uuuu', 'uuuu', 1527400576.0, 0, 0, 0, 0, 0), (17, 'ii@ii.com', 'ii', 'ii', 1527400832.0, 0, 0, 0, 0, 0), (18, 'jj@jj.com', 'jj', 'jj', 1527400832.0, 0, 0, 0, 0, 0), (19, 'ooo@ooo.com', 'ooo', 'ooo', 1527402240.0, 0, 0, 0, 0, 0), (20, 'pp@pp.com', 'pp', 'pp', 1527402240.0, 0, 0, 0, 0, 0), (21, 'ww@ww.com', 'ww', 'ww', 1527402624.0, 0, 0, 0, 0, 0), (22, '11510706@sustc.edu', 'yanxiangyisb', 'password', 1527403008.0, 0, 0, 0, 0, 0), (24, '11510225@sustc.edu', 'hu', 'password', 1

In [77]:
# test source
print(c.insert_source('test',description='test_desc'))
print(c.get_source_id(sourcename='d'))
print(c.get_source(sourcename='test_proj'))
print(c.get_all_source())
print(c.get_source_number())

0
None
(11, 'test_proj', 0, 1, 1527402240.0, 'this is a test project', 1, 1)
[(11, 'test_proj', 0, 1, 1527402240.0, 'this is a test project', 1, 1), (12, 'test', 3456, 1, 1527409408.0, 'test_desc', 1, 54321), (13, 'xiednproj', 0, 1, 1527928320.0, 'xiedn single option project', 2, 11), (14, 'yxy_test', 4, 2, 1527935232.0, 'This is a test for auto upload task.', 3, 11), (15, 'nb_test', 0, 2, 1528030464.0, 'This is a test for nb.', 1, 11)]
5


In [80]:
print(c.load_data('/home/se2018/data/test_proj/', sourcename='test_proj'))
print(c.get_textdataid(data_index=723154, sourcename='test_proj'))
#print(c.update_final_labelid(index=723154, labelid=1, souceid=11))

1
23


In [81]:
c.close()

In [299]:
now = datetime.datetime.now()
a = now - datetime.timedelta(days=8)

In [300]:
a = a.strftime("%s")

In [301]:
int(a)

1527923927

In [305]:
#l = c.get_user_mainpage_pan_history(user_email="xuch@sb.com")
next(x[0] for x in enumerate(l) if int(x[1][0])>int(a))


0

In [303]:
for ll in l:
    print(ll[0]>int(a))

True
True


In [304]:
l = [(1528612352.0,), (1527928320.0,)]

In [269]:
l

[(1528612352.0,), (1527928320.0,)]